### Seq2seq для машинного перевода

План на сегодня:
1. Токенизация текста: byte-pair encoding, sentencepiece
2. Encoder-decoder модель для перевода с немецкого на английский
3. Добавляем механизм внимания

In [ ]:
# ! pip install datasets transformers sentencepiece

In [ ]:
import torch
from torch import Tensor, nn
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

from transformers import T5Tokenizer

### 1. Готовим данные

In [ ]:
train_dataset = load_dataset("bentrevett/multi30k", split="train")
test_dataset = load_dataset("bentrevett/multi30k", split="test")

In [ ]:
train_dataset[0]

In [ ]:
import matplotlib.pyplot as plt


def length_histogram(dataset, ax, bins=20) -> None:
    en_lengths = []
    de_lengths = []
    for sample in dataset:
        en_lengths.append(len(sample["en"].split(" ")))
        de_lengths.append(len(sample["de"].split(" ")))

    ax.hist(en_lengths, alpha=0.5, bins=bins, label="en")
    ax.hist(de_lengths, alpha=0.5, bins=bins, label="de")
    ax.legend()


fig, axes = plt.subplots(1, 2, figsize=(10, 3))
length_histogram(train_dataset, axes[0])
length_histogram(test_dataset, axes[1])

Оставим только сравнительно короткие предложения, чтобы можно было чему-то научиться за короткое время

In [ ]:
maxlen = 7


def filter_dataset(dataset, maxlen: int) -> list[dict[str, str]]:
    return [
        dataset[i]
        for i in range(len(dataset))
        if len(dataset[i]["en"].split(" ")) <= maxlen
    ]


train_filtered = filter_dataset(train_dataset, maxlen)
test_filtered = filter_dataset(test_dataset, maxlen)

print(len(train_filtered), len(test_filtered))

#### 1.1. Токенизация: byte-pair encoding

Построение:

Начинаем со словаря, состоящего из отдельных символов (начальные токены).
На каждом шаге:
1. Оцениваем частоту всех пар токенов внутри слов, находим самую частую
2. Добавляем её в список токенов и в таблицу слияний
3. Останавливаемся, когда достигаем максимального размера словаря


Применение:

1. Разбиваем текст на символы
2. Находим первое возможное слияние в таблице и применяем его
3. Останавливаемся, когда дальнейшие слияния невозможны



<img src="https://lena-voita.github.io/resources/lectures/seq2seq/bpe/build_merge_table.gif" style="background:white" height="300"/>
<img src="https://lena-voita.github.io/resources/lectures/seq2seq/bpe/bpe_apply.gif" style="background:white" height="300"/>


Реализаций много, мы будем использовать токенизатор  из библиотеки `transformers`, где помимо самого подготовленного токенизатора (`sentencepiece.SentencePieceProcessor`) много полезных методов для кодирования и декодирования.

Добавим при создании новый токен, который будет указывать на начало перевода

In [ ]:
tokenizer: T5Tokenizer = T5Tokenizer.from_pretrained(
    "t5-small", padding_size="right", bos_token="</b>", legacy=False
)

In [ ]:
print("Размер словаря: ", len(tokenizer))

In [ ]:
encoded_german = tokenizer.encode(train_dataset[0]["de"])
encoded_english = tokenizer.encode(train_dataset[0]["en"])
print(encoded_german)
print(tokenizer.decode(encoded_german))
print(encoded_english)
print(tokenizer.decode(encoded_english))

Об упаковке в батчи можно больше не беспокоиться - токенизатор умеет обрабатывать сразу пачку примеров

In [ ]:
batch = [train_dataset[i]["en"] for i in range(4)]

encoded_batch = tokenizer.batch_encode_plus(
    batch, padding="longest", return_tensors="pt"
)
print(encoded_batch["input_ids"].shape)
print(encoded_batch.keys())

Возвращается два значения: `input_ids` - это наши токены, а `attention_mask` - это тензор, равный по размеру батчу токенов, где на месте `pad_token` стоят нули, в остальных позициях - единицы. Это нам понадобится потом.

А ещё можно кодировать сразу входные и выходные данные:

In [ ]:
inputs = [train_dataset[i]["en"] + tokenizer.bos_token for i in range(4)]
targets = [train_dataset[i]["de"] for i in range(4)]

encoded_batch = tokenizer(
    inputs, text_target=targets, padding="longest", return_tensors="pt"
)
print(encoded_batch.keys())

Используем это в `collate_fn` для сборки батчей:

In [ ]:
def collate_fn(
    tokenizer: T5Tokenizer, batch: list[tuple[str, str]]
) -> tuple[Tensor, Tensor]:
    prompt = tokenizer.bos_token
    inputs, targets = zip(*[(pair["de"], prompt + pair["en"]) for pair in batch])
    encoded_batch = tokenizer(
        inputs, text_target=targets, padding="longest", return_tensors="pt"
    )
    return encoded_batch

In [ ]:
batch = [train_dataset[i] for i in range(4)]
encoded_batch = collate_fn(tokenizer, batch)
print(encoded_batch.keys())

In [ ]:
print(encoded_batch["input_ids"].shape)
print(encoded_batch["attention_mask"].shape)
print(encoded_batch["labels"].shape)

Всё готово для получения минибатчей из датасетов:

In [ ]:
train_loader = DataLoader(
    train_filtered,
    batch_size=32,
    shuffle=True,
    collate_fn=lambda batch: collate_fn(tokenizer, batch),
)
test_loader = DataLoader(
    test_filtered,
    batch_size=32,
    shuffle=False,
    collate_fn=lambda batch: collate_fn(tokenizer, batch),
)

In [ ]:
batch = next(iter(train_loader))

### 2. Encoder-decoder модель для перевода на рекуррентных сетях

![img](https://esciencegroup.files.wordpress.com/2016/03/seq2seq.jpg)

Напишем энкодер, который будет возвращать последнее состояние

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size: int, hidden_dim: int) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)

    def forward(self, source: Tensor) -> Tensor:
        h = self.embedding(source)
        h, _ = self.rnn(h)
        return h

In [ ]:
encoder = Encoder(vocab_size=len(tokenizer), hidden_dim=128)
h = encoder.forward(batch["input_ids"])
print(h.shape)

Декодер использует это состояние в качестве собственного начального:

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size: int, hidden_dim: int) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRUCell(hidden_dim, hidden_dim)
        self.lm_head = nn.Linear(hidden_dim, vocab_size)

    def _get_last_encoder_state(
        self, encoder_states: Tensor, attention_mask: Tensor
    ) -> Tensor:
        B, T, _ = encoder_states.shape
        last_idx = attention_mask.sum(dim=-1) - 1
        return encoder_states[torch.arange(B), last_idx]

    def forward(
        self, encoder_states: Tensor, attention_mask: Tensor, target: Tensor
    ) -> Tensor:
        B, T = target.shape

        embeds = F.relu(self.embedding(target))
        h = self._get_last_encoder_state(encoder_states, attention_mask)
        logits = []
        for t in range(T):
            h = self.rnn.forward(embeds[:, t], h)
            logits.append(self.lm_head.forward(h))

        return torch.stack(logits, 1)

In [ ]:
decoder = Decoder(vocab_size=len(tokenizer), hidden_dim=128)
logits = decoder.forward(h, batch["attention_mask"], batch["labels"])
logits.shape

Попробуем обучить:

In [ ]:
import lightning as L
from lightning.pytorch.utilities.types import STEP_OUTPUT, OptimizerLRScheduler


class Seq2Seq(L.LightningModule):
    def __init__(
        self,
        encoder: Encoder,
        decoder: Decoder,
        tokenizer: T5Tokenizer,
        lr: float = 0.01,
    ) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.tokenizer = tokenizer
        self.lr = lr

    def forward(self, source: Tensor, attention_mask: Tensor, target: Tensor) -> Tensor:
        h = self.encoder.forward(source)
        logits = self.decoder.forward(h, attention_mask, target)
        return logits

    def training_step(self, batch: dict[str, Tensor], batch_idx: int) -> STEP_OUTPUT:
        logits = self.forward(
            batch["input_ids"], batch["attention_mask"], batch["labels"]
        )
        loss = F.cross_entropy(
            logits[:, :-1].reshape(-1, len(self.tokenizer)),
            batch["labels"][:, 1:].flatten(),
            ignore_index=self.tokenizer.pad_token_id,
        )
        self.log("loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self) -> OptimizerLRScheduler:
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def translate(
        self,
        input_ids: Tensor,
        attention_mask: Tensor,
        bos_token_id: int,
        max_new_tokens: int = 20,
    ) -> Tensor:
        h = self.encoder.forward(input_ids)
        idx = torch.full((input_ids.shape[0], 1), fill_value=bos_token_id)

        for t in range(max_new_tokens):
            logits = self.decoder.forward(h, attention_mask, idx)[:, -1]
            new_token = logits.argmax(dim=-1, keepdim=True)
            idx = torch.cat([idx, new_token], dim=1)

        return idx

In [ ]:
trainer = L.Trainer(accelerator="cpu", max_epochs=5)
encoder = Encoder(vocab_size=len(tokenizer), hidden_dim=128)
decoder = Decoder(vocab_size=len(tokenizer), hidden_dim=128)
seq2seq = Seq2Seq(encoder, decoder, tokenizer)
trainer.fit(model=seq2seq, train_dataloaders=train_loader)

In [ ]:
def translate_batch(batch: dict[str, Tensor], model: Seq2Seq, tokenizer: T5Tokenizer):
    source = batch["input_ids"]
    target = batch["labels"]

    preds = model.translate(
        source, batch["attention_mask"], tokenizer.bos_token_id, max_new_tokens=20
    )

    # decode

    source, target, preds = map(
        lambda x: tokenizer.batch_decode(x, skip_special_tokens=True),
        (source, target, preds),
    )

    for src, tgt, pred in zip(source, target, preds):
        print(f"Deutsch: {src}")
        print(f"English: {tgt}")
        print(f"Translation: {pred}\n")

In [ ]:
translate_batch(next(iter(train_loader)), seq2seq, tokenizer)

### Attetion

![img](https://i.imgur.com/6fKHlHb.png)

Реализуем слой аддитивного внимания

На вход: последовательность состояний энкодера $h_0^e, h_1^e, ..., h_T^e$ и текущее состояние декодера $h^d$

1. получим логиты для весов внимания с помощью двуслойного перцептрона: $$a_t = \psi(\tanh(\phi_e(h_t^e) + \phi_d(h_d)))$$
2. рассчитываем вероятности $$ p_t = {{e ^ {a_t}} \over { \sum_\tau e^{a_\tau} }} $$
3. считаем вектор контекста как взвешенную сумму состояний энкодера 
$$ c = \sum_t p_t \cdot h^e_t $$

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim: int) -> None:
        super().__init__()
        ...

    def forward(
        self, encoder_states: Tensor, attention_mask: Tensor, decoder_states: Tensor
    ) -> Tensor:
        B, T, d = encoder_states.shape

        ...

А теперь модифицируем наш декодер для использования механизма внимания

In [ ]:
class DecoderWithAttention(nn.Module):
    def __init__(self, vocab_size: int, hidden_dim: int) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRUCell(hidden_dim, hidden_dim)
        self.lm_head = nn.Linear(hidden_dim, vocab_size)

    def _get_last_encoder_state(
        self, encoder_states: Tensor, attention_mask: Tensor
    ) -> Tensor:
        B, T, _ = encoder_states.shape
        last_idx = attention_mask.sum(dim=-1) - 1
        return encoder_states[torch.arange(B), last_idx]

    def forward(
        self, encoder_states: Tensor, attention_mask: Tensor, target: Tensor
    ) -> Tensor:
        B, T = target.shape

        embeds = F.relu(self.embedding(target))
        h = self._get_last_encoder_state(encoder_states, attention_mask)
        logits = []
        for t in range(T):
            h = self.rnn.forward(embeds[:, t], h)
            logits.append(self.lm_head.forward(h))

        return torch.stack(logits, 1)

In [ ]:
encoder = Encoder(vocab_size=len(tokenizer), hidden_dim=128)
decoder = DecoderWithAttention(vocab_size=len(tokenizer), hidden_dim=128)
seq2seq_attention = Seq2Seq(encoder, decoder, tokenizer)
print(
    seq2seq_attention.forward(
        batch["input_ids"], batch["attention_mask"], batch["labels"]
    ).shape
)

In [ ]:
trainer = L.Trainer(accelerator="cpu", max_epochs=5)
trainer.fit(model=seq2seq_attention, train_dataloaders=train_loader)

In [ ]:
translate_batch(next(iter(train_loader)), seq2seq_attention, tokenizer)